In [1]:
# install packages to run from dash import dcc, html, Input, Output, no_update, Dash
# import plotly.graph_objects as go
# import pandas as pd
# import pathlib

!pip install dash
!pip install pandas
!pip install plotly
!pip install pathlib

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached blinker-1.7.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata (34 kB)
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.2 MB 11.6 MB/s eta 0:00:01
   - -------------------------------------- 0.5/10.2 MB 6.2 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/10.2 MB 7.4 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/10.2 MB 6.9 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.2 MB 6.6 MB/s eta 0:00:02
   ------- -------------------------------- 1.9/10.2 MB 7.0 MB/s eta 0:00:02
   -------- ------------------------------- 2.2/10.2 MB 7.2 MB/s eta 0:00:02
   --------- ------------------------------ 2.5/10

In [ ]:
# activate the virtual environment by running the following command in the terminal
# source venv/bin/activate

In [2]:
!pip install rdkit

   ---------------------------------------- 0.0/21.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/21.1 MB ? eta -:--:--
   ---------------------------------------- 0.2/21.1 MB 2.4 MB/s eta 0:00:09
    --------------------------------------- 0.5/21.1 MB 3.9 MB/s eta 0:00:06
   - -------------------------------------- 1.0/21.1 MB 5.1 MB/s eta 0:00:04
   -- ------------------------------------- 1.4/21.1 MB 6.5 MB/s eta 0:00:04
   --- ------------------------------------ 1.7/21.1 MB 6.4 MB/s eta 0:00:04
   ---- ----------------------------------- 2.1/21.1 MB 6.8 MB/s eta 0:00:03
   ---- ----------------------------------- 2.4/21.1 MB 6.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.9/21.1 MB 7.2 MB/s eta 0:00:03
   ------ --------------------------------- 3.3/21.1 MB 7.4 MB/s eta 0:00:03
   ------- -------------------------------- 3.8/21.1 MB 7.6 MB/s eta 0:00:03
   -------- ------------------------------- 4.3/21.1 MB 7.9 MB/s eta 0:00:03
   --------- 

In [3]:
#check version of pandas
!pip show dash

Name: dash
Version: 2.16.1
Summary: A Python framework for building reactive web-apps. Developed by Plotly.
Home-page: https://plotly.com/dash
Author: Chris Parmer
Author-email: chris@plotly.com
License: MIT
Location: c:\Users\cypri\OneDrive - Imperial College London\Year 5\T1\Cyprien_master_project\InteractiveAppMsci\.venv\Lib\site-packages
Requires: dash-core-components, dash-html-components, dash-table, Flask, importlib-metadata, nest-asyncio, plotly, requests, retrying, setuptools, typing-extensions, Werkzeug
Required-by: dash-bootstrap-components, dash-tools


In [6]:
# launch the app
!python .\src\app.py

^C


: 

In [6]:
morgan_keys.columns

Index(['InChIKey', 'Morgan_Fingerprint', 'Cluster', 'Oligomers', 'Counts',
       'smiles', 'X_frag_mol', 'mean_diff_SchNet_IP_weighted',
       'mean_diff_PaiNN_IP_weighted', 'mean_diff_Equiformer_IP_weighted',
       'mean_diff_SphereNet_IP_weighted',
       'mean_diff_DimeNetPlusPlus_IP_weighted',
       'mean_diff_DimeNet_IP_weighted', 'mean_diff_SchNet_fosc1_weighted',
       'mean_diff_PaiNN_fosc1_weighted', 'mean_diff_Equiformer_fosc1_weighted',
       'mean_diff_SphereNet_fosc1_weighted',
       'mean_diff_DimeNetPlusPlus_fosc1_weighted',
       'mean_diff_DimeNet_fosc1_weighted', 'mean_diff_SchNet_ES1_weighted',
       'mean_diff_PaiNN_ES1_weighted', 'mean_diff_Equiformer_ES1_weighted',
       'mean_diff_SphereNet_ES1_weighted',
       'mean_diff_DimeNetPlusPlus_ES1_weighted',
       'mean_diff_DimeNet_ES1_weighted', 'mean_diff_average_fosc1_weighted',
       'mean_diff_average_IP_weighted', 'mean_diff_average_ES1_weighted',
       'Similarity_Score', 'img', 'MW', 'CT', 'pca1'

In [28]:
feature_path = r'C:\Users\cypri\OneDrive - Imperial College London\Year 5\T1\Cyprien_master_project\InteractiveAppMsci\data\interactive_results.pkl'
morgan_keys = pd.read_pickle(feature_path)


In [34]:
!pip install Chem

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB 8.6 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 7.4 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.5 MB 6.6 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 6.5 MB/s eta 0:00:01
   --------------------------- ------------ 1.1/1.5 MB 5.1 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 4.7 MB/s eta 0:00:01
   -------------------------------- ------- 1.2/1.5 MB 4.1 MB/s eta 0:00:01
   -------------------

In [37]:
from rdkit.Chem import Draw
import base64
import io
from io import BytesIO
from tqdm import tqdm
from PIL import Image
from rdkit import Chem

def get_images(smiles_list: list[str], legend: str):
    images = []
    for i, smiles in enumerate(tqdm(smiles_list, desc='Generating images')):
        rdmol = Chem.MolFromSmiles(smiles)
        d2d = Draw.MolDraw2DCairo(300, 300)
        dopts = d2d.drawOptions()
        dopts.clearBackground = False
        d2d.DrawMolecule(rdmol, legend=legend)
        d2d.FinishDrawing()
        drawing_text = d2d.GetDrawingText()
        bio = BytesIO(drawing_text)
        image = Image.open(bio)
        buffer = io.BytesIO()
        image.save(buffer, format="png")
        encode_image = base64.b64encode(buffer.getvalue()).decode()
        im_url = f"data:image/png;base64, {encode_image}"
        images.append(im_url)
    return images

In [42]:
# turn the smiles into images
images = get_images(morgan_keys['smiles'], 'Molecule')
morgan_keys['images_url'] = images

Generating images: 100%|██████████| 310/310 [00:05<00:00, 60.47it/s]


In [46]:
# save morgan_keys to pickle
morgan_keys.to_pickle(r'C:\Users\cypri\OneDrive - Imperial College London\Year 5\T1\Cyprien_master_project\InteractiveAppMsci\data\interactive_results.pkl')

In [57]:
from dash import Dash, dcc, html, Input, Output, no_update, callback
import plotly.graph_objects as go
import pandas as pd

# Load Morgan keys data
feature_path = r'C:\Users\cypri\OneDrive - Imperial College London\Year 5\T1\Cyprien_master_project\InteractiveAppMsci\data\interactive_results.pkl'
morgan_keys = pd.read_pickle(feature_path)

# Create scatter plot
fig = go.Figure(data=[
    go.Scatter(
        x=morgan_keys['pca1'],
        y=morgan_keys['pca2'],
        mode="markers",
        marker=dict(
            color=morgan_keys['Cluster'],  # Color by Cluster number
            colorscale='viridis',  # Use viridis color scale
            size=10,
            colorbar={"title": "Cluster"},
            line={"color": "#444"},
            opacity=0.8,
        ),
    )
])

fig.update_layout(
    xaxis=dict(title='PCA1'),
    yaxis=dict(title='PCA2'),
    plot_bgcolor='rgba(255,255,255,0.1)',
    width=800,
    height=800,
)

# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Graph(id="graph", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])


@callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph", "hoverData"),
)

def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update
    
    # demo only shows the first point, but other points may also be available
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]
    im_url = morgan_keys['images_url'][num]

    children = [
        html.Div(children=[
            html.Img(src=im_url, style={"width": "100%"}),
        ],
        style={'width': '200px', 'white-space': 'normal'})
    ]

    return True, bbox, children


if __name__ == "__main__":
    app.run_server(debug=True)


In [78]:
from dash import Dash, dcc, html, Input, Output, no_update, callback
import plotly.graph_objects as go
import pandas as pd

# Load Morgan keys data
feature_path = r'C:\Users\cypri\OneDrive - Imperial College London\Year 5\T1\Cyprien_master_project\InteractiveAppMsci\data\interactive_results.pkl'
morgan_keys = pd.read_pickle(feature_path)

# Define list of y-axis targets and models
targets = ['IP', 'ES1', 'fosc1']
models = ['average', 'SchNet', 'DimeNet', 'DimeNetPlusPlus', 'PaiNN', 'Equiformer', 'SphereNet']
x_axis = ['CT', 'MW', 'Similarity_Score', 'Cluster']

# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Graph(id="graph", clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
    dcc.Dropdown(
        id='y-axis-target-dropdown',
        options=[{'label': target, 'value': target} for target in targets],
        value=targets[0],
        clearable=False,
    ),
    dcc.Dropdown(
        id='x-axis-dropdown',
        options=[{'label': x, 'value': x} for x in x_axis],
        value=x_axis[0],
        clearable=False,
    ),
    html.Div([
        html.Button(id=f'y-axis-model-button-{model}', n_clicks=0, children=model, style={'margin-right': '10px'})
        for model in models
    ]),
])

# Define callback to update the graph
@app.callback(
    Output("graph", "figure"),
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph", "hoverData"),
    Input("x-axis-dropdown", "value"),
    Input("y-axis-target-dropdown", "value"),
    *[Input(f'y-axis-model-button-{model}', 'n_clicks') for model in models],
)
def update_graph(hoverData, x_axis, y_target, *n_clicks):
    # Get selected models
    selected_models = [model for model, clicks in zip(models, n_clicks) if clicks % 2 != 0]

    # Filter Morgan keys data based on selected x_axis and y_target
    filtered_data = morgan_keys[[x_axis] + [f'mean_diff_{model}_{y_target}_weighted' for model in selected_models]]

    # Create scatter plot
    fig = go.Figure()
    for model in selected_models:
        y_column = f'mean_diff_{model}_{y_target}_weighted'
        fig.add_trace(go.Scatter(
            x=filtered_data[x_axis],
            y=filtered_data[y_column],
            mode="markers",
            marker=dict(
                color=morgan_keys['Cluster'], 
                colorscale='viridis', 
                colorbar={"title": "Cluster"}, 
                size=10,
                line={"color": "#444"},
                opacity=0.8,
            ),
            name=y_column
        ))

    fig.update_layout(
        xaxis=dict(title=f'{x_axis}'),
        yaxis=dict(title=f'MAE_{y_target}'),
        plot_bgcolor='rgba(255,255,255,0.1)',
        width=800,
        height=400,
        )

    # Determine if tooltip should be shown
    if hoverData is None:
        show_tooltip = False
        bbox = no_update
        children = no_update
    else:
        # Demo only shows the first point, but other points may also be available
        pt = hoverData["points"][0]
        bbox = pt["bbox"]
        num = pt["pointNumber"]
        im_url = morgan_keys['images_url'][num]

        children = [
            html.Div(children=[
                html.Img(src=im_url, style={"width": "100%"}),
            ],
            style={'width': '200px', 'white-space': 'normal'})
        ]

        show_tooltip = True

    return fig, show_tooltip, bbox, children


if __name__ == "__main__":
    app.run_server(debug=True)


In [92]:
# view dash_boothstrap_components version
!pip show dash_bootstrap_components

Name: dash-bootstrap-components
Version: 1.6.0
Summary: Bootstrap themed components for use in Plotly Dash
Home-page: https://dash-bootstrap-components.opensource.faculty.ai/
Author: Faculty
Author-email: opensource@faculty.ai
License: Apache Software License
Location: c:\Users\cypri\OneDrive - Imperial College London\Year 5\T1\Cyprien_master_project\InteractiveAppMsci\.venv\Lib\site-packages
Requires: dash
Required-by: dash-tools


In [91]:
from dash import Dash, dcc, html, Input, Output, no_update
import plotly.graph_objects as go
import pandas as pd
import dash_bootstrap_components as dbc

# Load Morgan keys data
feature_path = r'C:\Users\cypri\OneDrive - Imperial College London\Year 5\T1\Cyprien_master_project\InteractiveAppMsci\data\interactive_results.pkl'
morgan_keys = pd.read_pickle(feature_path)
morgan_keys['Cluster'] = morgan_keys['Cluster'].astype('category')

# Create PCA scatter plot
pca_fig = go.Figure(data=[
    go.Scatter(
        x=morgan_keys['pca1'],
        y=morgan_keys['pca2'],
        mode="markers",
        marker=dict(
            color=morgan_keys['Cluster'],  # Color by Cluster number
            colorscale='viridis',  # Use viridis color scale
            size=10,
            colorbar={"title": "Cluster"},
            line={"color": "#444"},
            opacity=0.8,
        ),
    )
])

pca_fig.update_layout(
    xaxis=dict(title='PCA1'),
    yaxis=dict(title='PCA2'),
    plot_bgcolor='rgba(255,255,255,0.1)',
    width=800,
    height=400,
    title='Reduced dimensionality visualisation of fragments using PCA'
)

# Initialize Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define app layout
app.layout = html.Div([
    dbc.Row([
        dbc.Col(
            dcc.Graph(id="pca-graph", figure=pca_fig, clear_on_unhover=True)
        )
    ]),
    dbc.Row([
        dbc.Col(
            html.Div([
                dcc.Graph(id="graph", clear_on_unhover=True),
                dcc.Tooltip(id="graph-tooltip"),
                dcc.Dropdown(
                    id='y-axis-target-dropdown',
                    options=[{'label': target, 'value': target} for target in targets],
                    value=targets[0],
                    clearable=False,
                ),
                dcc.Dropdown(
                    id='x-axis-dropdown',
                    options=[{'label': x, 'value': x} for x in x_axis],
                    value=x_axis[0],
                    clearable=False,
                ),
                html.Div([
                    html.Button(id=f'y-axis-model-button-{model}', n_clicks=0, children=model, style={'margin-right': '10px'})
                    for model in models
                ]),
            ]),
            width=12
        )
    ])
])

# Define callback to update the graph
@app.callback(
    Output("graph", "figure"),
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph", "hoverData"),
    Input("pca-graph", "hoverData"),
    Input("x-axis-dropdown", "value"),
    Input("y-axis-target-dropdown", "value"),
    *[Input(f'y-axis-model-button-{model}', 'n_clicks') for model in models],
)
def update_graph(graph_hoverData, pca_hoverData, x_axis, y_target, *n_clicks):
    # Get selected models
    selected_models = [model for model, clicks in zip(models, n_clicks) if clicks % 2 != 0]

    # Filter Morgan keys data based on selected x_axis and y_target
    filtered_data = morgan_keys[[x_axis] + [f'mean_diff_{model}_{y_target}_weighted' for model in selected_models]]

    # Create scatter plot
    fig = go.Figure()
    for model in selected_models:
        y_column = f'mean_diff_{model}_{y_target}_weighted'
        fig.add_trace(go.Scatter(
            x=filtered_data[x_axis],
            y=filtered_data[y_column],
            mode="markers",
            marker=dict(
                color=morgan_keys['Cluster'], 
                colorscale='viridis', 
                colorbar={"title": "Cluster"}, 
                size=10,
                line={"color": "#444"},
                opacity=0.8,
            ),
            name=y_column
        ))

    fig.update_layout(
        xaxis=dict(title=f'{x_axis}'),
        yaxis=dict(title=f'MAE_{y_target}'),
        plot_bgcolor='rgba(255,255,255,0.1)',
        width=800,
        height=400,
        title='Model Performance Comparison'
    )

    # Determine if tooltip should be shown
    if graph_hoverData is None:
        show_tooltip = False
        bbox = no_update
        children = no_update
    
    if pca_hoverData is None:
        show_tooltip = False
        bbox = no_update
        children = no_update

    if graph_hoverData is not None:
        # Demo only shows the first point, but other points may also be available
        pt = graph_hoverData["points"][0]
        bbox = pt["bbox"]
        num = pt["pointNumber"]
        im_url = morgan_keys['images_url'][num]

        children = [
            html.Div(children=[
                html.Img(src=im_url, style={"width": "100%"}),
            ],
            style={'width': '200px', 'white-space': 'normal'})
        ]

        show_tooltip = True

    if pca_hoverData is not None:
        # Demo only shows the first point, but other points may also be available
        pt = pca_hoverData["points"][0]
        bbox = pt["bbox"]
        num = pt["pointNumber"]
        im_url = morgan_keys['images_url'][num]

        children = [
            html.Div(children=[
                html.Img(src=im_url, style={"width": "100%"}),
            ],
            style={'width': '200px', 'white-space': 'normal'})
        ]

        show_tooltip = True

    return fig, show_tooltip, bbox, children


if __name__ == "__main__":
    app.run_server(debug=True)
